<img src="https://drive.google.com/uc?id=1-hPP-XPm9_5M3orUgmompcVleQ5xvPST" style="Width:1000px">

# The Nutritionist Recommender Systems

<img src="https://drive.google.com/uc?id=1TED1hS7Vf7XvMSZGq8PP74yPqyDduBhd" style="width:900px">


Recommender systems are everywhere, and you use them daily: when you follow the recommendation of **Netflix** for what to watch, when you are presented with ads on social media, and whenever you shop online and an option to check what *others with similar interests* have bought comes up. Recommender are big business, and in this exercise, we will build one!

Obviously, our system will be far simpler than the sophisticated systems built by **Big Tech**, but the principle will be the same, and it will rely on `unsupervised learning`. If you want a more in-depth look at recommenders, and another possible exercise to do, <a href="https://blog.codecentric.de/recommender-system-movie-lens-dataset"> check this excellent online tutorial</a>.

Here, we will use the `KNN` algorithm to obtain a recommendation for similar food items based on their nutritional value. The data comes from `Kaggle` and can be <a href="https://www.kaggle.com/datasets/thedevastator/the-nutritional-content-of-food-a-comprehensive">accessed here</a>.

I hear you say "But wait! KNN is a supervised algorithm!". Yup. But by the end of this exercise, you will know how we can use it as an unsupervised recommender.

## Opening and exploring the data

Load the data located in the file `food.csv` and save it in a dataframe named `data`. 

In [ ]:
from nbta.utils import download_data
download_data(id='10xedwItyYpILipNP4xrK4ywAvAumfrP7')

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

# Explore the dataset

Do the following:
* Have a look at the different features, and their range, using `describe()`
* Check if there are any missing values, and take note of where these are
* Look at the data type for each column
* Do a histogram plot of all of the numerical variables, and assess if they are overall normally distributed (I recommend using a large `figsize`, for instance `(30,30)`: otherwise it will be hard to see the distribution of your data well)

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

# Prepare the dataset

By following the steps above, you must have come to the following concusions:

* The `Category` and `Description` columns are categorical objects, and describe the food item. Thus, an excellent label that we will use with our recommender.
* There is a `Nutrient Data Bank Number`: this is a unique ID for each food item, but it does not bring any valuable data to our food items.
* The only missing values are in the `Data.Household Weights.1st Household Weight Description` and `Data.Household Weights.2nd Household Weight Description`. These are also the only `string` (or `pd.object`) types other than the item description mentioned above.
* The data is most definitely not normally distributed: we tend fo have a lot of items with low values, and few with very high values.

Let's explore the implications of some of these observations.

## Categorical data with `KNN`

Our recommender will be based on `KNN`: this implies that it will rely on the `Euclidian distance` between each item to make a decision. As with each algorithm using `Euclidian distance`, we cannot and should not use categorical data encoded through a `OneHotEncoder` or other encoders that do not respect a distance metric. There are alternative algorithms, for instance you can <a href="https://www.analyticsvidhya.com/blog/2021/06/kmodes-clustering-algorithm-for-categorical-data/">check K-Mode clustering</a> as a replacement for `K-Means clustering` when you deal with categorical data.

But here we will simply stick to numerical data for our recommender.

## Log-Normal distribution

Many variables that have a distribution that looks similar to the one you have plotted above often have a `log-normal` distribution. That is to say, if you take the logarithm of the value, the distribution of that variable is normal (or quasi-normal).

For machine learning, a common feature engineering technique is thus to take the log of the value, rather than the value itself. This spreads the data more evenly across a range of value, which is beneficial to distance-based algorithms such as `KNN`.

Beware however that many values might be equal to `zero`, and the log of `zero` is not mathematically possible. The trick is to add a very small value to the actual feature (say, `1e-5`) to signiffy that this is below detection limit.

## Do the following

Given the consideration above, you should do the following to prepare your data:
1. Create a `np.array` that contains the name of the columns we want to use for our recommender (call it `features`). This must include all the numerical columns, with the exception of `Nutrient Data Bank Number` (as mentioned above), as well as the `Data.Household Weights.1st Household Weight` and `Data.Household Weights.2nd Household Weight`: we have missing values in these columns, and they are not relevant for our task. What we want is only the `nutritional values` of each food, as we will try to recommend similar foods.
2. Using a combination of `Pipeline`, `ColumnTransformer` and `FunctionTransformer`, create a `preproc` object capable of taking the raw data reindexed (your `data`), select only the `features` we need, and preparing it for machine learning. This also implies that your `preproc` will do a `log-transform` of the data, and will then scale it using a `MinMaxScaler`.
3. `fit()` your `preproc` to the `data`, and create the `X_prep` array by using the `transform` function of your `preproc`. Turn `X_prep` into a `pd.DataFrame` with the relevant original columns from `data` in order to be able to visualize it well (and for convenience)
4. Plot a new `hist()` of all of the features of `X_prep`: hopefully you should see that although the data is not perfect (there are still many samples with zero values), the non-zero values are much more normally distributed than before.

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('data_preparation',
                         data = X_prep
)

result.write()
print(result.check())

# Building the Basic Recommender

So how can we use a `KNN` as a recommender? Well, you need to remember that when we `fit()` a `KNN` (irrespective of whether it is a `KNeighborsRegressor` or a `KNeighborsClassifer`), the algorithm needs to `compute the Euclidian distance between each pair of samples in the database`. So in effect, once fitted, the trained algorithm can be used in an unsupervised manner to find samples that are closest to any other sample in the training set. By the way, this is also why `KNN` tends to be inefficient to large datasets: the number of pairwise you need to compute grows exponentially.

So, for a simple `KNN` recommender, we need to do the following:

1. Create a new `KNN` regressor, and fit it with you `X_prep` as the `X`, and any feature from `X_prep` to act as a token `y`. Remember, we don't care about predicting `y`: all we want is to access the pre-computed distance in the feature space between two features. 
2. Once the algorithm is fitted, these distances are saved and can be accessed through the `kneighbors` function.

Go ahead and create a new `KNeighborsRegressor` and save it as a new variable called `recommender`. Then, fit it to the `X_prep` and any feature you want (I used `Data.Alpha Carotene` but it is not important which one you choose).

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

## Making recommendations

Now, let's see how we can make a recommendation. First, access the food item at `index=22` in `data`, and see what it is (**hint**: this one makes Swiss people melt...):

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

You should see that this is the index for Gruyère, and it returns the description of this cheese as well as all of the nutritional values of this food.

Now, let's find the 5 foods in our `KNN` that are most closely related to Gruyère:
1. Using `X_pred`, search for the item at `index=22`. Because `X_pred` is the transformed version of `data`, their index are similar so this will return the transformed nutritional values for Gruyère. Save this under a named variable, say, `food`.
2. Use the `food` variable as an input to the `kneighbors` function of your `recommender` object, alongside the number of neighbors we want (5). If in doubt, <a href="https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html">consult the documentation</a> to see how this works.
3. The above function should return a tupple of two arrays: the first array containst the Euclidian distance between your sample and the three nearest neighbors. Note that the first distance is zero, i.e. this actually the Gruyère Cheese. This is confirmed when you look at the second array, which contains the index at which the items are location. As you can see, the index of the first item is indeed the index of Gruyère!
4. Using the second array return, you can now retrieve from `data` the names and all of the nutritional values of the three closest items in our `KNN` "database" of distance. Save this as a variable named `similar_foods`.

Does the recommendation make sense?

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('initial_recommender',
                         nb_recommendations = similar_foods.shape[0],
                         first_recommendation = similar_foods.Description.values[0],
                         second_recommendation = similar_foods.Description.values[1],
)

result.write()
print(result.check())

# Building our Recommender Class

Now you know the magic behind distance-based recommender: I find them cool, but there is nothing magic to it, really. Let's take this to a new level, and write a `Python class` to make intelligent recommendations about food items for us. I wrote for you the skeleton of the `NutritionRecommender` class into the file `nutrition_recommender.py`. This is a golden opportunity to practice `object oriented` programming in Python whilst building something fun.

Here is what your class should be able to do:

### When an instance of the class is created:

* The instance automatically open the `food.csv` file, and an instance variable called `data` is created
* The file is prepared via a similar `pipeline` as what you did before, and an `X_prep` instance variable is created
* A `recomender` KNN instance variable is also automatically created, and fitted to the `X_prep`

All of the above, you have done before in this notebook. This needs to happen when the `__init__()` function is called, but you can spread the logic of your class constructor (i.e. how a class construct an instance) into different functions to keep your code clean: all you need is to make sure to call these functions within `__init__()`

### Obtaining a list of food item that contains a work or string

One thing that is currently difficult is to find a particular food item. For instance, how do I find where **"Brie"** is if I want to find similar cheeses? You will write a method called `description_contains(self, substring)` that will do the following:
* Return a dataframe of all the items in `data` that contains substring

For instance, passing `"cheese"` as an input to this function should return 139 items, starting with "CHEESE,BLUE". To make your life simple, make sure that the function is handling different capitalization.

### Obtaining `n` similar food

The function `find_similar(self, item, nb_recommendations=5)` does two things:
1. If `item` is of type `int`, then it looks for item at index `item` in `X_prep` and returns a dataframe containing `nb_recommendations` rows, and only the following useful columns: `Category`,`Description`,`Data.Kilocalories`. Note that the item itself is not return, so it needs to be `nb_recommendations` other than the original item (which is what a user would expect).
2. If `item` is of type `string`, then the strategy is different: the function calls the `self.description_contains` function with `item` as argument, and use the first row of the returned dataframe as the item to lookup. It then proceed to do as in point `1` above.



In [ ]:
from nutrition_recommender import NutritionRecommender

recommender = NutritionRecommender()

In [ ]:
best_similarity = recommender.find_similar(105,5)

### 🧪 Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('recommender', best_similarity=best_similarity)

result.write()
print(result.check())

# 🏁 Finished!

Well done! <span style="color:teal">**Push your exercise to GitHub**</span>, and move on to the next one.